In [8]:
import tensorflow as tf
import numpy as np 
import pandas as pd 
from data_loader import bench, squat, deadlift

In [9]:
class RepCount(tf.keras.Model):
    def __init__(self):
        super(RepCount, self).__init__()
        self.values = tf.constant([0], dtype=tf.float32)

    def heaviside_custom(self, x, values):
        ones = tf.ones_like(x)
        zeros = tf.zeros_like(x)
        return tf.where(x > 0, ones, tf.where(x < 0, zeros, values))

    def call(self, tens):
        t = self.heaviside_custom(tens, self.values)
        b = t[1:] - t[:-1]
        mask = tf.reduce_any(tf.not_equal(b, 0), axis=1)
        b = tf.boolean_mask(b, mask)
        b = tf.where(tf.equal(b, -1), tf.zeros_like(b), b)
        return tf.reduce_sum(b, axis=0)

model = RepCount()
dummy_input = tf.constant((bench[0][['gravityX', 'gravityY', 'gravityZ']]).dropna().to_numpy(), dtype=tf.float32)
output = model(dummy_input)
print("Output:", output)

Output: tf.Tensor([ 1.  1. 13.], shape=(3,), dtype=float32)


In [10]:
saved_model_dir = "models/model2"
tf.saved_model.save(model, saved_model_dir)


INFO:tensorflow:Assets written to: models/model2/assets


INFO:tensorflow:Assets written to: models/model2/assets


In [11]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open("rep_count_model.tflite", "wb") as f:
    f.write(tflite_model)


W0000 00:00:1741807061.384770 7799673 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1741807061.384781 7799673 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-12 12:17:41.384926: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: models/model2
2025-03-12 12:17:41.385071: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-12 12:17:41.385076: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: models/model2
2025-03-12 12:17:41.386087: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-12 12:17:41.388760: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: models/model2
2025-03-12 12:17:41.390834: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 5909 microseconds.
2025-03-12 12:17:41.403685: I tensorflow/compiler/mlir/

In [18]:
dummy_input = tf.constant((bench[1][['gravityX', 'gravityY', 'gravityZ']]).dropna().to_numpy(), dtype=tf.float32)
print(dummy_input.shape[0] % 128)
dummy_input = tf.reduce_mean(dummy_input)


interpreter = tf.lite.Interpreter(model_path="rep_count_model.tflite")
interpreter.allocate_tensors()


input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:", input_details)
print("Output details:", output_details)

input_shape = input_details[0]['shape']


21
Input details: [{'name': 'serving_default_inputs:0', 'index': 0, 'shape': array([4174,    3], dtype=int32), 'shape_signature': array([4174,    3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'PartitionedCall:0', 'index': 29, 'shape': array([3], dtype=int32), 'shape_signature': array([3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [6]:
input_shape

array([4174,    3], dtype=int32)

In [7]:
interpreter.resize_tensor_input(input_details[0]['index'], dummy_input.shape)
interpreter.allocate_tensors()  
interpreter.set_tensor(input_details[0]['index'], dummy_input)
interpreter.invoke()


RuntimeError: Given shapes, [1173,3], [4174,3] and [1], are not broadcastable.Node number 4 (SELECT_V2) failed to prepare.

In [ ]:
output_details = interpreter.get_output_details()
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Model output:", output_data)


In [20]:
a = tf.constant([[1, 1], [1, 1]])

In [21]:
a

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[1, 1],
       [1, 1]], dtype=int32)>

In [29]:
tf.pad(a, a)

<tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[0, 0, 0, 0],
       [0, 1, 1, 0],
       [0, 1, 1, 0],
       [0, 0, 0, 0]], dtype=int32)>